In [30]:
import pandas as pd
import requests
import json
import random
import openai

URL_BASE_API_SDW = "https://sdw-2023-prd.up.railway.app/"
openai.api_key = pd.read_csv("../credentials/credentials.csv")["gpt_key"][0]

In [31]:
# Função para buscar dados dos clientes/usuarios
def get_user(id):
    response = requests.get(f"{URL_BASE_API_SDW}users/{id}")

    return response.json() if response.status_code == 200 else None

In [32]:
# Busca os usuarios que existem dentro da API, para não trabalharmos com usuários não cadastrados
valid_users = requests.get(f'{URL_BASE_API_SDW}users')

# Seleciona aleatoriamente, 10 IDs para o projeto
ids = random.sample([x["id"] for x in valid_users.json()],2)

In [33]:
# Busca dados de cada usuário
users =  [get_user(x) for x in ids]

In [34]:
# Função responsável por criar mensagem de marketing para base de clientes
def generate_marketing(user):
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "system", 
            "content": "Você é um gerente de marketing da empresa de celular Samsung"},
            {"role": "user",
             "content": f"Crie uma mensagem de marketing personalizada para {user['name']} (resposta limitada a 20 palavras)"}
        ]
    )

    return completion.choices[0].message["content"].strip('"')

In [35]:
# Loop que usa função para gerar mensagem e atualizar dados.
for user in users:
    news = generate_marketing(user)
    user['news'].append({
        "icon":"https://github.com/hrmdamasceno/dio-prj01-mensagem-gpt/blob/2b43cc9e3a3274b0e0d062d37d1c5586ad474c24/data/phone.svg",
        "description": news}
    )

In [36]:
# Função para realizar carregamento das mensagens geradas
def update_user(user):
    response = requests.put(f"{URL_BASE_API_SDW}users/{user['id']}", json=user)
    return True if response.status_code == 200 else (False, response.status_code)

In [37]:
# Utiliza função para realizar um update nos dados
for user in users:
    success = update_user(user)
    print(f"User {user['name']} was updated? {success}")

User Pyterson was updated? True
User lalala was updated? True
